In [6]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, AveragePooling2D,
                                   GlobalAveragePooling2D, BatchNormalization,
                                   Activation, Add, Concatenate, Dense, Dropout,
                                   Multiply, Reshape, Permute)
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, models

In [8]:

import tensorflow as tf

dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/plantvillage-dataset/color',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(64, 64),
    batch_size=16,
)

dataset_val = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/plantvillage-dataset/color',
    validation_split=0.2,
    subset="validation",
    
    seed=123,
    image_size=(64,64),
    batch_size=16,
)



Found 54305 files belonging to 38 classes.
Using 43444 files for training.


I0000 00:00:1751299891.218677      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 54305 files belonging to 38 classes.
Using 10861 files for validation.


In [9]:

normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = dataset_train.map(lambda x, y: (normalization_layer(x), y))
val_ds = dataset_val.map(lambda x, y: (normalization_layer(x), y))

In [10]:


def simple_attention(x, reduction_ratio=8):
    
    channel = x.shape[-1]

    # Global Average Pooling
    gap = GlobalAveragePooling2D()(x)
    gap = layers.Reshape((1, 1, channel))(gap)

    attention = layers.Dense(channel // reduction_ratio, activation='relu')(gap)
    attention = layers.Dense(channel, activation='sigmoid')(attention)

    return Multiply()([x, attention])

In [11]:
list=[]
def residual_block(x, filters, stride=1):
    
    shortcut = x

    x = Conv2D(filters, 3, strides=stride, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = Conv2D(filters, 3, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)

    # Adjust shortcut if needed
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, 1, strides=stride, use_bias=False)(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x
def residual_block_group(x, filters, n_blocks, stride=1):
    x = residual_block(x, filters, stride=stride)
    for _ in range(1, n_blocks):
        x = residual_block(x, filters, stride=1)
    return x


def dense_block(x, num_layers, growth_rate):
    
    concat_features = [x]

    for _ in range(num_layers):
        x = Concatenate()(concat_features)
        out = BatchNormalization()(x)
        out = Activation('relu')(out)
        out = Conv2D(4 * growth_rate, (1, 1), padding='same', use_bias=False)(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
        out = Conv2D(growth_rate, (3, 3), padding='same', use_bias=False)(out)
        concat_features.append(out)

    x = Concatenate()(concat_features)
    return x

def transition_layer(x):
    
    x = BatchNormalization()(x)
    x = Conv2D(x.shape[-1] // 2, (1, 1), padding='same', use_bias=False)(x)
    x = AveragePooling2D(pool_size=(2, 2), strides=2)(x)
    return x

def gfa_residual_stream(input_tensor):

    global list
    x = Conv2D(64, 3, padding='same', use_bias=False)(input_tensor)

    x = BatchNormalization()(x)
    x = layers.ReLU()(x)

    


    x = residual_block_group(x, filters=128, n_blocks=2, stride=1)
    list.append(x)
    x = residual_block_group(x, filters=256, n_blocks=2, stride=2)
    list.append(x)
    x = residual_block_group(x, filters=512, n_blocks=2, stride=2)
    list.append(x)



    x = simple_attention(x)


    return GlobalAveragePooling2D()(x)
def gfa_parallel(list):
  pooled_list = []
  for i in list:
    i = simple_attention(i)
    pooled_list.append(GlobalAveragePooling2D()(i))
  list = Concatenate()(pooled_list)
  return list





def sf_dense_stream(input_tensor, growth_rate=16):
    
    x = Conv2D(64, (3, 3), padding='same', use_bias=False)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)


    x = dense_block(x, num_layers=4, growth_rate=growth_rate)
    x = transition_layer(x)

    x = dense_block(x, num_layers=4, growth_rate=growth_rate)
    x = transition_layer(x)

    x = dense_block(x, num_layers=8, growth_rate=growth_rate)

    sf_pool = GlobalAveragePooling2D()(x)
    return sf_pool







def build_derefnet(input_shape=(64, 64, 3), num_classes=38, growth_rate=16):
    
    from tensorflow.keras import Input
    inputs = Input(shape=input_shape)


    
    gfa_out = gfa_residual_stream(inputs)  
    sf_out = sf_dense_stream(inputs, growth_rate)
    gfa_parallel_out=gfa_parallel(list)


    fused = Concatenate()([gfa_out, sf_out, gfa_parallel_out])
    fused = Dropout(0.5)(fused)

    
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(fused)

    return Model(inputs, outputs, name='DeReFNet_PlantVillage')

In [12]:

print(tf.config.list_logical_devices())



[LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]


In [14]:


model = build_derefnet(input_shape=(64,64,3), num_classes=38, growth_rate=16)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:


model.fit(train_ds, validation_data=val_ds, epochs=30)


Epoch 1/30


I0000 00:00:1751299980.328046     101 service.cc:148] XLA service 0x7ad528003500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751299980.328718     101 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1751299984.342809     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/2716 ━━━━━━━━━━━━━━━━━━━━ 50:45:47 67s/step - accuracy: 0.0000e+00 - loss: 3.7766

I0000 00:00:1751300008.475970     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2716/2716 ━━━━━━━━━━━━━━━━━━━━ 328s 96ms/step - accuracy: 0.5554 - loss: 1.5820 - val_accuracy: 0.6810 - val_loss: 1.1258
Epoch 2/30
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 230s 85ms/step - accuracy: 0.8048 - loss: 0.6169 - val_accuracy: 0.7612 - val_loss: 0.8234
Epoch 3/30
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 221s 81ms/step - accuracy: 0.8657 - loss: 0.4198 - val_accuracy: 0.8055 - val_loss: 0.6184
Epoch 4/30
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 225s 83ms/step - accuracy: 0.8962 - loss: 0.3238 - val_accuracy: 0.6941 - val_loss: 1.4264
Epoch 5/30
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 228s 84ms/step - accuracy: 0.9168 - loss: 0.2613 - val_accuracy: 0.6560 - val_loss: 1.8528
Epoch 6/30
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 221s 81ms/step - accuracy: 0.9306 - loss: 0.2132 - val_accuracy: 0.8541 - val_loss: 0.4951
Epoch 7/30
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 222s 82ms/step - accuracy: 0.9359 - loss: 0.1928 - val_accuracy: 0.7874 - val_loss: 0.9987
Epoch 8/30
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 221s 81ms/step - accuracy: 0.9468 - lo